In [1]:
import numpy as np
import pandas as pd
import sqlite3 

## Data preprocessing
Contains methods to:
1. Get the csv student attendance.
2. Extract session information from columns.
3. Clean the original data i.e., remove NaNs, correct data types.

In [2]:
class PreprocessStudentAttendance:
    def __init__(self, folder_name, file_name):
        self.folder = folder_name
        self.file = file_name 

    def get_session_details(clm):
        '''
        Extract details of sessions from the columnn names
        Input:
            clm: the columnn names, list type 
        Return: 
            df_details: the session details, dataframe type
        '''
        # separate details
        df_details = pd.DataFrame(clm, columns= ["details"])
        df_details = df_details["details"].str.split("\n", expand= True)  
        df_details.columns = [
            "semester_week", "date", "time",
            "lecture_type", "room_id", "EMPTY"
        ] 

        # further separate columns by deliminators
        old_clms = ["semester_week", "date", "time"]
        new_clms = [
            ["semester","week"], 
            ["day", "date"], 
            ["start_time", "end_time"]
        ]
        delims = [".", " ", "-"]

        for c in range(len(old_clms)):
            df_details[new_clms[c]] = df_details[old_clms[c]].str.split(
                delims[c], 
                expand= True
            )

        new_clms = ["date", "date", "semester", "week", "room_id"]
        delims = ["(", ")", "S", "W", "..."]
        for c in range(len(new_clms)):
            df_details[new_clms[c]] = df_details[new_clms[c]].str.strip(delims[c])
        
        # format to datetime columns SQL standard:
        # https://www.sqlite.org/lang_datefunc.html
        time_clm = ["start_time", "end_time"]
        for t in time_clm:
            day = df_details["date"].str[:2]
            month = df_details["date"].str[3:5]
            year = df_details["date"].str[6:10]
            time = df_details[t].str.strip() + ":00"

            df_details[t] = year + "-" + month + "-" + day + " " + time
            
        # add session number column (used to join later)
        idx = 0
        df_details.insert(loc= idx, column= "session", value= df_details.index+1)

        # remove redundant columns 
        df_details.drop(["semester_week", "time", "EMPTY"],
            axis = 1, 
            inplace = True
        )

        return df_details

    def get_student_attendance_csv(self):
        '''
        Reads .csv and returns the sessions 
        and student attendance per session.
        Input:
            Student attendance details from csv files
        Return:
            df_student_attendance
            df_sessions
        '''
        df_student_attendance = pd.read_csv(
            "./"+ self.folder+ "/"+self.file+".csv", 
            index_col= 0
        )
        clm = df_student_attendance.columns.to_list()
        df_sessions = PreprocessStudentAttendance.get_session_details(clm)

        df_student_attendance.columns = np.arange(
            len(df_student_attendance.columns)
        )
        return (df_sessions, df_student_attendance)

    def clean_student_attendance(self, df_student_atten):
        '''
        Cleans student attendance data by:
            + Replace the original strings with boolean or None
            + Remove NaN rows and columns 
        Input: 
            df_student_atten
        Return: 
            df_student_atten (cleaned)
        '''
        df_student_atten.index.names = ["sid"] # renames index (student id = "sid")
        df_student_atten.columns += 1

        old_vals = ['Ex', 'GPS', 'X']
        new_vals = [None, True, False]
            
        for v in range(len(old_vals)):
            df_student_atten.replace(old_vals[v], new_vals[v], inplace = True)

        df_student_atten.dropna(
            axis = 0, 
            how = 'all',
            inplace = True
        ) # drops rows with NaN 
        df_student_atten.dropna(
            axis = 1,
            how = 'all', 
            inplace = True
        ) # drops columns with NULL
        return df_student_atten

In [3]:
# files =  ["22COA111ModuleRegister", "22COA122ModuleRegister"]
# for file in files:
#     # print(get_student_attendance_csv(folder, file))

## Database operations 
Contains methods to:
1. Connect to database (and check it it has been connected).
2. Write dataframe to existing/newly created database.
3. Replace table if it already exists in the database with the latest version.
4. Get session and student attendance from database.
5. Add a new session to the database.
6. Add new student attendance to the database.

In [4]:
'''
*I found changing the type for datatypes other than 'INTEGER' in python does 
not guarentee the type is maintainted in SQLite with 'to_sql' alone.
It is instead stored as a storage class rather than a specific datatype:
https://www.sqlite.org/datatype3.html
Datatypes can be concretely changed using this function with sqlalchemy:
https://www.sqlalchemy.org/
'''
class SQL3:
    def __init__(self, database_name, table_name, df_student_data):
        self.db = database_name
        self.table = table_name
        self.df = df_student_data

    def connect_to_db(self):
        '''
        Connects to an SQlite database.
        If the database does not exist it will be created.
        Input: 
            database_name (self.db)
        Return:
            conn
        '''
        try: # try connecting to the database
            conn = sqlite3.connect(self.db)
            return conn
        except Exception as Error:
            print(Error) # if it fails return the error
            if conn is not None:
                conn.close()
        
    def write_df_to_db(self):
        '''
        Write new dataframe to database if it does not already exist.
        Input:
            table_name (self.table)
            df_student_data (self.df)
        Return:
            None (Dataframe is written into database.)
        '''
        conn = SQL3.connect_to_db(self)
        if conn is not None:
            cur = conn.cursor()
            try:
                # create table with correct datatypes*
                if "session" in self.df.columns:
                    cur.execute (
                        "CREATE TABLE " + self.table + ''' (
                        session INTEGER,
                        date DATE,
                        lecture_type VARCHAR, 
                        room_id VARCHAR, 
                        semester INTEGER, 
                        week INTEGER, 
                        day VARCHAR, 
                        start_time DATETIME, 
                        end_time DATETIME);
                        '''
                    )
                    self.df.to_sql(
                        name= self.table,
                        con= conn,
                        if_exists= "replace",
                        index= False
                    )
                else:
                    self.df.to_sql(
                        name= self.table,
                        con= conn,
                        if_exists= "replace",
                        index= False
                    )
            except sqlite3.OperationalError:
                SQL3.replace_sql_tbl(self)
            print("Dataframe has been successfully written to SQL!")
        else:
            print("Connection to database failed. Please try again.")

    def replace_sql_tbl(self):
        '''
        If the SQL table already exists 
        the table is replaced with its latest version.
        Input:
            table_name (self.table)
            df_student_data (self.df)
        Return:
            None (Existing sql table is replaced.)
        '''
        conn = SQL3.connect_to_db(self)
        if conn is not None:
            cur = conn.cursor()
            cur.execute("PRAGMA foreign_keys = off;")
            cur.execute("BEGIN TRANSACTION;")
            cur.execute("ALTER TABLE " + self.table + ''' RENAME
                TO ''' + self.table + "_old;")
            cur.execute(
                "CREATE TABLE " + self.table + ''' (
                session INTEGER,
                date DATE,
                lecture_type VARCHAR, 
                room_id VARCHAR, 
                semester INTEGER, 
                week INTEGER, 
                day VARCHAR, 
                start_time DATETIME, 
                end_time DATETIME); 
                '''
            )
            # writes new data to the table created
            self.df.to_sql(
                name= self.table,
                con= conn,
                if_exists= "append",
                index= False
            )
            cur.execute("DROP TABLE " + self.table + "_old;")
            cur.execute("PRAGMA foreign_keys = on;")
            conn.commit()
        else:
            print("Connection to database failed. Please try again.")

    def get_attendance_details_from_db(self):
        '''
        Reads the database and returns student attendance details
        Input:
            table_name (self.table)
        Output:
            df_atten_details 
        '''
        conn = SQL3.connect_to_db(self)
        if conn is not None:
            cur = conn.cursor()
            df_atten_details = pd.read_sql("SELECT * FROM " + self.table, 
                con= conn
            )
            return df_atten_details
        else:
            print("Connection to database failed. Please try again.")
#==============================================================================

    # def add_student_attendance_to_db(): 

    # def add_session_to_db():

    # def remove_student_attendance():

    # def remove_session():
#==============================================================================

In [5]:
# Testing: 
folder = "cop504cwdata"
file = "22COA111ModuleRegister"

pp = PreprocessStudentAttendance(folder, file)
df_sessions, df_stu_atten = pp.get_student_attendance_csv()
df_clean_stu_atten = pp.clean_student_attendance(df_stu_atten)
df_clean_stu_atten
df_sessions

database = "CWDatabase.db"
table = "COA111Sessions" 
sql3 =  SQL3(
    database_name= database, 
    table_name= table, 
    df_student_data= df_sessions
)
sql3.connect_to_db()
sql3.write_df_to_db() # it also replaces table if it already exists
sql3.get_attendance_details_from_db()

Dataframe has been successfully written to SQL!


,session,date,lecture_type,room_id,semester,week,day,start_time,end_time
0,1,03-10-2022,Lecture,U020,1,1,Monday,2022-10-03 09:00:00,2022-10-03 10:00:00
1,2,05-10-2022,Lecture,SMB014,1,1,Wednesday,2022-10-05 10:00:00,2022-10-05 11:00:00
2,3,12-10-2022,Lecture,SMB014,1,2,Wednesday,2022-10-12 10:00:00,2022-10-12 11:00:00
3,4,12-10-2022,Tutorial,,1,2,Wednesday,2022-10-12 12:00:00,2022-10-12 13:00:00
4,5,13-10-2022,Lecture,CC011,1,2,Thursday,2022-10-13 14:00:00,2022-10-13 15:00:00
5,6,19-10-2022,Lecture,SMB014,1,3,Wednesday,2022-10-19 10:00:00,2022-10-19 11:00:00
6,7,19-10-2022,Tutorial,,1,3,Wednesday,2022-10-19 12:00:00,2022-10-19 13:00:00
7,8,20-10-2022,Lecture,CC011,1,3,Thursday,2022-10-20 14:00:00,2022-10-20 15:00:00
8,9,26-10-2022,Lecture,SMB014,1,4,Wednesday,2022-10-26 10:00:00,2022-10-26 11:00:00
9,10,26-10-2022,Tutorial,,1,4,Wednesday,2022-10-26 12:00:00,2022-10-26 13:00:00


# Testing

Run code (functions)

In [6]:
# Write test into a function

Desiging GUI for Sections 1.3 to 1.5.
Get ideas: https://www.youtube.com/watch?v=M43u_K_ZZfQ
User options:
1. Add csv files by BROWSING button (looks for file in computer)
2. Look up student id (a. show weekly attendancre graph, )
3. Look up module code and week number 
4. Add new student attendance info (show updated colour coded files (white if normal, red if disengaged etc.))
5. Add new session info  (")
6. Remove "
7. Remove "
8. Extra: Option to save student record as a pdf 